In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('cleaned_data.csv')

In [3]:
#create new feature combining 3 columns num outpatients, num emergencies, num inpatients

df['hospital utilisation'] = df['num_outpatient_appointments'] + df['num_previous_emergencies'] + df['num_inpatient_overnight_stays']

In [4]:
df['readmitted'] = df['readmitted'].replace('>30', 1)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [5]:
df['change_in_meds'] = df['change_in_meds'].replace('Ch', 1)
df['change_in_meds'] = df['change_in_meds'].replace('No', 0)

In [6]:
to_drop = ['citoglipton','examide']

df.drop(to_drop, inplace=True, axis=1)


In [7]:
print('no readmission', df['readmitted'][df['readmitted']==0].count())

no readmission 53316


In [8]:
df['diabates_med_prescribed'] = df['diabates_med_prescribed'].replace('No', 0)
df['diabates_med_prescribed'] = df['diabates_med_prescribed'].replace('Yes', 1)
df['gender'] = df['gender'].replace('Male', 0)
df['gender'] = df['gender'].replace('Female', 1)

#replace age code intervals 0- 10: 1, 10-20:2 etc
for i in range (0,10):
    df['age']=df['age'].replace('['+str(10*i)+ '-' +str(10*(i+1))+')', i+1)
df.head(3)

,Unnamed: 0,race,gender,age,days_in_hospital,num_lab_procedures,num_not_lab_procedures,num_current_medications,num_outpatient_appointments,num_previous_emergencies,...,insulin,glyburide_metformin,glipizide_metformin,glimepiride_pioglitazone,metformin_rosiglitazone,metformin_pioglitazone,change_in_meds,diabates_med_prescribed,readmitted,hospital utilisation
0,0,Caucasian,1,1,1,41,0,1,0,0,...,not_prescribed,not_prescribed,not_prescribed,not_prescribed,not_prescribed,not_prescribed,0,0,0,0
1,1,Caucasian,1,2,3,59,0,18,0,0,...,dosage_increased,not_prescribed,not_prescribed,not_prescribed,not_prescribed,not_prescribed,1,1,1,0
2,2,AfricanAmerican,1,3,2,11,5,13,2,0,...,not_prescribed,not_prescribed,not_prescribed,not_prescribed,not_prescribed,not_prescribed,0,1,0,3


In [9]:
#new feature for changes in made in each patient regardless of drug
meds = ['metformin', 'nateglinide', 'chlorpropamide', 'glimepiride',
        'acetohexamide', 'glipizide','repaglinide','glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone',
        'acarbose', 'miglitol', 'troglitazone','tolazamide','insulin',  'glyburide_metformin',
        'glipizide_metformin', 'glimepiride_pioglitazone', 
        'metformin_rosiglitazone', 'metformin_pioglitazone']


In [10]:
for col in meds:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'not_prescribed' or x == 'no_change_dosage') else 1)
df['numchange'] = 0
for col in meds:
    colname = str(col) + 'temp'
    df['numchange'] = df['numchange'] + df[colname]
    del df[colname]
    
df['numchange'].value_counts()  

0    72347
1    25735
2     1298
3      107
4        5
Name: numchange, dtype: int64

In [11]:
for col in meds:
    df[col]=df[col].replace('not_prescribed', 0)
    df[col]=df[col].replace('no_change_dosage', 0)
    df[col]=df[col].replace('dosage_increased', 1)
    df[col]=df[col].replace('dosage_decreased', 1)

In [12]:
column_names1 =['race']
new_values = {'AfricanAmerican': 0, 'Asian':1, 'Caucasian': 3, 'Hispanic': 4, 'Other': 5 }
for medicine in column_names1:
    df[medicine] = df[medicine].replace(new_values)

In [26]:
X = df.drop('readmitted', axis = 1)
y = df.readmitted
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)


In [25]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
print (logmodel.score(X,y))

0.535882281992522


In [23]:
predictions=logmodel.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.53      1.00      0.70     15958
           1       0.00      0.00      0.00     13890

    accuracy                           0.53     29848
   macro avg       0.27      0.50      0.35     29848
weighted avg       0.29      0.53      0.37     29848



/Users/Susan/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[13269,     0],
       [11604,     0]])

In [31]:
to_drop = ['num_not_lab_procedures']

df.drop(to_drop, inplace=True, axis=1)


In [32]:
X = df.drop('readmitted', axis = 1)
y = df.readmitted
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)


In [33]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
print (logmodel.score(X,y))

0.535882281992522


In [37]:
from sklearn.tree import DecisionTreeClassifier
dtree =DecisionTreeClassifier()

In [38]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [39]:
predict =dtree.predict(X_test)

In [41]:
from sklearn.metrics import classification_report,confusion_matrix

In [42]:
print(confusion_matrix(y_test,predict))
print('\n')
print(classification_report(y_test,predict))

[[7682 5587]
 [5637 5967]]


              precision    recall  f1-score   support

           0       0.58      0.58      0.58     13269
           1       0.52      0.51      0.52     11604

    accuracy                           0.55     24873
   macro avg       0.55      0.55      0.55     24873
weighted avg       0.55      0.55      0.55     24873



In [43]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
rfc = RandomForestClassifier(n_estimators = 100)

In [73]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [68]:
rfc_pred= rfc.predict(X_test)

In [69]:
print(confusion_matrix(y_test,rfc_pred))
print('\n')
print(classification_report(y_test,rfc_pred))

[[9227 4042]
 [5432 6172]]


              precision    recall  f1-score   support

           0       0.63      0.70      0.66     13269
           1       0.60      0.53      0.57     11604

    accuracy                           0.62     24873
   macro avg       0.62      0.61      0.61     24873
weighted avg       0.62      0.62      0.62     24873

